# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,la passe,45.5549,-0.8967,25.13,74,73,5.40,FR,1691690970
1,1,susuman,62.7833,148.1667,7.05,74,2,0.84,RU,1691691159
2,2,callao,-12.0667,-77.1500,23.18,68,100,6.17,PE,1691691159
3,3,saldanha,-33.0117,17.9442,15.88,71,0,3.46,ZA,1691691159
4,4,yellowknife,62.4560,-114.3525,19.29,55,75,3.60,CA,1691691072


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = 'Humidity',
    color = 'City'
)

# Display the map
city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_cities = city_data_df[(city_data_df['Max Temp'] > 21) & (city_data_df['Max Temp'] < 27)]
filtered_cities = filtered_cities[filtered_cities['Wind Speed'] < 4.5]
filtered_cities = filtered_cities[filtered_cities['Cloudiness'] == 0]

# Drop any rows with null values
filtered_cities = filtered_cities.dropna()

# Display sample data
filtered_cities

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
52,52,noshiro,40.2039,140.0242,23.37,81,0,3.42,JP,1691691164
61,61,mossel bay,-34.1831,22.1460,21.96,66,0,0.37,ZA,1691691165
67,67,kolonia,50.9333,6.9500,24.42,47,0,1.03,DE,1691690988
140,140,kalemie,-5.9475,29.1947,23.13,59,0,1.72,CD,1691691173
184,184,al khums,32.6486,14.2619,25.74,63,0,3.73,LY,1691691177
252,252,chingirlau,51.0950,54.0814,24.44,33,0,3.95,KZ,1691691184
290,290,vadso,70.0744,29.7487,25.49,73,0,3.09,NO,1691691189
340,340,isheyevka,54.4167,48.2667,21.73,38,0,0.42,RU,1691691195
345,345,biarritz,43.4801,-1.5556,24.70,73,0,3.09,FR,1691691195
348,348,colonia,50.9333,6.9500,24.90,47,0,1.03,DE,1691691129


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_cities.copy()
hotel_df = hotel_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
52,noshiro,JP,40.2039,140.0242,81,
61,mossel bay,ZA,-34.1831,22.1460,66,
67,kolonia,DE,50.9333,6.9500,47,
140,kalemie,CD,-5.9475,29.1947,59,
184,al khums,LY,32.6486,14.2619,63,
252,chingirlau,KZ,51.0950,54.0814,33,
290,vadso,NO,70.0744,29.7487,73,
340,isheyevka,RU,54.4167,48.2667,38,
345,biarritz,FR,43.4801,-1.5556,73,
348,colonia,DE,50.9333,6.9500,47,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel

radius = 10000
params = {
    'categories': 'accommodation.hotel',
    'apiKey': geoapify_key,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
noshiro - nearest hotel: あづま旅館
mossel bay - nearest hotel: Protea Hotel Mossel Bay
kolonia - nearest hotel: Wasserturm Hotel Cologne
kalemie - nearest hotel: Hotel du Lac
al khums - nearest hotel: فندق الخمس
chingirlau - nearest hotel: No hotel found
vadso - nearest hotel: Scandic Vadsø
isheyevka - nearest hotel: No hotel found
biarritz - nearest hotel: Louisiane
colonia - nearest hotel: Wasserturm Hotel Cologne
bemidji - nearest hotel: Hampton Inn and Suites
lewiston orchards - nearest hotel: Holiday Inn Express Lewiston
ryotsu-minato - nearest hotel: ゆたかや旅館
holualoa - nearest hotel: Kona Hotel
bodo - nearest hotel: Zefyr hotell
suances - nearest hotel: Hotel Surfantes
plettenberg bay - nearest hotel: Milkwood Manor
aktobe - nearest hotel: Fresh


,City,Country,Lat,Lng,Humidity,Hotel Name
52,noshiro,JP,40.2039,140.0242,81,あづま旅館
61,mossel bay,ZA,-34.1831,22.1460,66,Protea Hotel Mossel Bay
67,kolonia,DE,50.9333,6.9500,47,Wasserturm Hotel Cologne
140,kalemie,CD,-5.9475,29.1947,59,Hotel du Lac
184,al khums,LY,32.6486,14.2619,63,فندق الخمس
252,chingirlau,KZ,51.0950,54.0814,33,No hotel found
290,vadso,NO,70.0744,29.7487,73,Scandic Vadsø
340,isheyevka,RU,54.4167,48.2667,38,No hotel found
345,biarritz,FR,43.4801,-1.5556,73,Louisiane
348,colonia,DE,50.9333,6.9500,47,Wasserturm Hotel Cologne


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display
# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = 'Humidity',
    hover_cols = ['City', 'Country', 'Hotel Name'],
    color = 'City'
)

# Display the map
hotel_map


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)